# Creating dataframes/csv to work with

## Begin wrangling data and combining the datasets

In [1]:
import pandas as pd
import numpy as np

In [75]:
# import raw datasets
genome_tags = pd.read_csv("data/genome-tags.csv")
genome_scores = pd.read_csv("data/genome-scores.csv")
cage_movies = pd.read_csv("data/nic-cage.csv")
movies = pd.read_csv("data/movies.csv")

tags = pd.read_csv("data/tags.csv")
ratings = pd.read_csv("data/ratings.csv")

## Standardizing and fitting cage_movies titles and matching them to movieId in MovieLens dataset

In [86]:
# remove trailing spaces in entire dataframe
movies = movies.applymap(lambda x: str(x).rstrip())

# split the title and year information from the title
titles = movies['title'].str.replace(' \([0-9]*\)$', '', regex = True)
years = movies['title'].str.extract(r'(\([0-9]*\)$)', expand = False) # regex capture group to get the suffix year appended to every movie title

titles

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
62418                             We
62419             Window of the Soul
62420                      Bad Poems
62421                   A Girl Thing
62422        Women of Devil's Island
Name: title, Length: 62423, dtype: object

In [103]:
# get rid of parens around year
years = years.str.replace('(', '', regex = True)
years = years.str.replace(')', '', regex = True)

years

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
62418    2018
62419    2001
62420    2018
62421    2001
62422    1962
Name: title, Length: 62423, dtype: object

In [104]:
# check for titles which didn't have years or in which the year wasn't extract properly
years.isna().sum()  # there are 412 titles without years

412

In [107]:
years.str.contains('\(').sum() + years.str.contains('\)').sum() # there are no years that still have appended parens

0

In [108]:
# put titles and years columns from above into movies dataframe
movies.title = titles
movies['year'] = years

# remove the trailing spaces in entire dataframe
movies = movies.applymap(lambda x: str(x).rstrip())

# get the ", The" at the end of movies[title] and append to front
movies.loc[movies['title'].str.contains(', The$'),'title'] = 'The ' + movies['title']
movies['title'] = movies['title'].str.replace(', The$', '', regex = True)
movies['title'] = movies['title'].str.replace(',The$', '', regex = True)

# remove the trailing spaces in entire dataframe
movies = movies.applymap(lambda x: str(x).rstrip())

# rearrange columns
movies = movies[['movieId', 'title', 'year', 'genres']]

movies.head()

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy


In [109]:
movies.describe()

,movieId,title,year,genres
count,62423,62423,62423,62423
unique,62423,58158,136,1639
top,126586,Cinderella,2015,Drama
freq,1,16,2513,9056


In [116]:
movies.head()

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy


In [117]:
# movies.isna() # this won't give NA values because the year has been changed to a string and the NaN becomes 'nan' string
movies.year.str.contains('nan').sum() # here are the 412 movies that don't have years

412

In [119]:
# drop the rows where the year is not provided because we need it to join with the nicolas cage movie dataset
# get names of indices where column year has value 'nan'
ind = movies[movies.year == 'nan'].index

# delete these row indices from movies dataframe
movies.drop(ind, inplace = True)

movies.year.str.contains('nan').sum() # returns 0 - successful!

0

In [120]:
# set types for movies columns
movies.movieId = movies.movieId.astype(int)
movies.year = movies.year.astype(int)

movies.dtypes

movieId     int64
title      object
year        int64
genres     object
dtype: object

In [121]:
# rename the cage_movies columns to lowercase
cage_movies = cage_movies.rename(columns = {'Movie': 'movie', 'Rating': 'rating', 'Character': 'character', 'Voice': 'voice_flag', 'Year': 'year', 'RottenTomatoes': 'rottentomatoes'})

# "Sorcerer's Apprentice" is misspelled 'Sorceror'
cage_movies['movie'] = cage_movies['movie'].str.replace('Sorceror', 'Sorcerer')

# Remove period from "Adaption."
cage_movies['movie'] = cage_movies['movie'].str.replace('Adaptation.', 'Adaptation', regex = False)

# "Gone in 60 Seconds" is spelled out 'Sixty' in cage_movies
cage_movies['movie'] = cage_movies['movie'].str.replace('Sixty', '60')

# "Amos & Andrew" is spelled "Amos and Andrew" in cage_movies
cage_movies['movie'] = cage_movies['movie'].str.replace('Amos and', 'Amos &')

# "Time to Kill" needs the Italian suffix in cage_movies
cage_movies['movie'] = cage_movies['movie'].str.replace('Time to Kill', 'Time to Kill (Tempo di uccidere)')

# "Best of Times" needs 'The ' prefixed in cage_movies
cage_movies['movie'] = cage_movies['movie'].str.replace('Best of Times', 'The Best of Times')

cage_movies

,movie,rating,character,voice_flag,year,rottentomatoes
0,A Score to Settle,NR,Frank Pierce,0,2019,13
1,Spider-Man: Into the Spider-Verse,PG,Spider-Man Noir,1,2018,97
2,Between Worlds,R,Joe,0,2018,X
3,Teen Titans Go! To the Movies,PG,Superman,1,2018,91
4,211,R,Mike Chandler,0,2018,5
...,...,...,...,...,...,...
86,Racing with the Moon,PG,Nicky,0,1984,60
87,Rumble Fish,R,Smokey,0,1983,70
88,Valley Girl,R,Randy,0,1983,82
89,Fast Times at Ridgemont High,R,Brad's Bud,0,1982,78


In [122]:
cage_movies.describe()

,voice_flag,year
count,91.000000,91.000000
mean,0.076923,2003.296703
std,0.267946,11.280755
min,0.000000,1981.000000
25%,0.000000,1994.000000
50%,0.000000,2006.000000
75%,0.000000,2013.500000
max,1.000000,2019.000000


In [123]:
# cage_movies.RottenTomatoes = cage_movies.RottenTomatoes.astype(int) - Returns error because there are RottenTomatoes ratings that are labeled 'X'

# return a list of movies that have incorrect RottenTomatoes rankings
cage_movies[cage_movies.rottentomatoes == 'X'].movie.drop_duplicates()

2                                        Between Worlds
10                              Vengeance: A Love Story
74    Industrial Symphony No. 1: The Dream of the Br...
77                     Time to Kill (Tempo di uccidere)
78                                     Never on Tuesday
83                                      The Boy in Blue
90                                    The Best of Times
Name: movie, dtype: object

In [124]:
# manually fix the RottenTomatoes rankings for the titles that have them
cage_movies.loc[cage_movies['movie'] == 'Between Worlds', 'rottentomatoes'] = 29

# so this is dangerous, but i want to include this movie because it's so fucking bonkers
cage_movies.loc[cage_movies['movie'] == 'The Best of Times', 'rottentomatoes'] = -1

# had to resort to audience score for this one because critics didn't rate it (travesty)
cage_movies.loc[cage_movies['movie'] == 'Time to Kill (Tempo di uccidere)', 'rottentomatoes'] = 23

# also dangerous but it was an avante-garde musical play directed by DAVID LYNCH
cage_movies.loc[cage_movies['movie'].str.contains('Industrial Symphony No. 1: The Dream of'), 'rottentomatoes'] = -1

# had to resort to audience score
cage_movies.loc[cage_movies['movie'] == 'Vengeance: A Love Story', 'rottentomatoes'] = 28

# audience score
cage_movies.loc[cage_movies['movie'] == 'The Boy in Blue', 'rottentomatoes'] = 45

# check to see if it took
cage_movies[cage_movies.rottentomatoes == 'X'].movie.drop_duplicates()

78    Never on Tuesday
Name: movie, dtype: object

In [125]:
# left join the movie information to the cage_movies dataset using title as key
cage_movies.movie = cage_movies.movie.astype(str)
movies.title = movies.title.astype(str)

# left join the movie information to the cage_movies dataset using title as key
cage = pd.merge(cage_movies, movies, left_on = ['movie', 'year'], right_on = ['title', 'year'], how = 'left')

In [126]:
# check for NaN values
print(cage[cage.isnull().any(axis=1)])

movie rating                     character  \
27  Ghost Rider: Spirit of Vengeance  PG-13  Johnny Blaze and Ghost Rider   
53                 A Christmas Carol     PG                        Marley   
70                     Red Rock West      R                       Michael   
78                  Never on Tuesday      R         Man in Red Sports Car   
79                    Vampire's Kiss      R                    Peter Loew   
90                 The Best of Times  TV-NR                      Nicholas   

    voice_flag  year rottentomatoes  movieId title genres  
27           0  2011             18      NaN   NaN    NaN  
53           1  2001             17      NaN   NaN    NaN  
70           0  1993             95      NaN   NaN    NaN  
78           0  1989              X      NaN   NaN    NaN  
79           0  1988             61      NaN   NaN    NaN  
90           0  1981             -1      NaN   NaN    NaN  


In [23]:
# Never on Tuesday is NA but it isn't included in the movies dataset so delete it from entire dataset
cage = cage.dropna()
cage = cage.drop(columns = ['title', 'title_trimmed'])

print(cage[cage.isnull().any(axis=1)])

KeyError: "['title' 'title_trimmed'] not found in axis"

In [28]:
# rearrange columns
cage = cage[['movieId', 'movie', 'year', 'rating', 'character', 'voice_flag', 'rottentomatoes', 'genres']]
cage.head()

,movieId,movie,year,rating,character,voice_flag,rottentomatoes,genres
0,204656,A Score to Settle,2019,NR,Frank Pierce,0,13,Action|Drama|Thriller
1,195159,Spider-Man: Into the Spider-Verse,2018,PG,Spider-Man Noir,1,97,Action|Adventure|Animation|Sci-Fi
2,194991,Between Worlds,2018,R,Joe,0,29,Action|Mystery|Thriller
3,191687,Teen Titans Go! To the Movies,2018,PG,Superman,1,91,Animation|Children
4,187713,211,2018,R,Mike Chandler,0,5,(no genres listed)


In [29]:
cage.describe()
# uh oh the count increased to 111

,year,voice_flag
count,111.000000,111.000000
mean,2004.504505,0.072072
std,10.898602,0.259780
min,1981.000000,0.000000
25%,1995.000000,0.000000
50%,2007.000000,0.000000
75%,2014.000000,0.000000
max,2019.000000,1.000000


In [34]:
# see what the duplicates are for the movies
cage.drop_duplicates()

,movieId,movie,year,rating,character,voice_flag,rottentomatoes,genres
0,204656,A Score to Settle,2019,NR,Frank Pierce,0,13,Action|Drama|Thriller
1,195159,Spider-Man: Into the Spider-Verse,2018,PG,Spider-Man Noir,1,97,Action|Adventure|Animation|Sci-Fi
2,194991,Between Worlds,2018,R,Joe,0,29,Action|Mystery|Thriller
3,191687,Teen Titans Go! To the Movies,2018,PG,Superman,1,91,Animation|Children
4,187713,211,2018,R,Mike Chandler,0,5,(no genres listed)
...,...,...,...,...,...,...,...,...
107,26521,Racing with the Moon,1984,PG,Nicky,0,60,Comedy|Drama|Romance
108,26485,Rumble Fish,1983,R,Smokey,0,70,Drama
109,6638,Valley Girl,1983,R,Randy,0,82,Comedy|Romance
110,3210,Fast Times at Ridgemont High,1982,R,Brad's Bud,0,78,Comedy|Drama|Romance


In [48]:
# set and double check types
cage.movieId = cage.movieId.astype(int)
cage.rottentomatoes = cage.rottentomatoes.astype(int)
cage.dtypes

Movie             object
Rating            object
Character         object
Voice              int64
Year               int64
RottenTomatoes     int64
movieId            int64
genres            object
dtype: object

In [74]:
# export cage to csv
path = '/Users/Kevin/cage-search/data/cleaned/cage_movies.csv'
cage.to_csv(path, index = False)

## create genres dataset with each movie and associated movie

In [57]:
cage.genres.drop_duplicates()

0                     Action|Drama|Thriller
1         Action|Adventure|Animation|Sci-Fi
2                   Action|Mystery|Thriller
3                        Animation|Children
4                        (no genres listed)
                       ...                 
85                             Comedy|Crime
93                           Comedy|Romance
97     Crime|Drama|Mystery|Romance|Thriller
100                   Comedy|Fantasy|Horror
106                           Crime|Musical
Name: genres, Length: 63, dtype: object

In [70]:
# split the genres up by the pipe delimiter and stack them using movieId as the index
genres = pd.DataFrame(cage.genres.str.split('|').tolist(), index = cage.movieId).stack()
genres

movieId   
204656   0       Action
         1        Drama
         2     Thriller
195159   0       Action
         1    Adventure
                ...    
3210     0       Comedy
         1        Drama
         2      Romance
26566    0       Comedy
         1        Drama
Length: 277, dtype: object

In [71]:
# get rid of the secondary index (which is essentially counting the number of genres per movieId)
genres = genres.reset_index([0, 'movieId'])
genres

,movieId,0
0,204656,Action
1,204656,Drama
2,204656,Thriller
3,195159,Action
4,195159,Adventure
...,...,...
272,3210,Comedy
273,3210,Drama
274,3210,Romance
275,26566,Comedy


In [72]:
# rename the genres columns, set their types, and get the movie title form the cage dataset defined above
genres.columns = ['movieId', 'genre']
genres = pd.merge(genres, cage[['movieId', 'movie']], on = 'movieId', how = 'left')
genres

,movieId,genre,Movie
0,204656,Action,A Score to Settle
1,204656,Drama,A Score to Settle
2,204656,Thriller,A Score to Settle
3,195159,Action,Spider-Man: Into the Spider-Verse
4,195159,Adventure,Spider-Man: Into the Spider-Verse
...,...,...,...
272,3210,Comedy,Fast Times at Ridgemont High
273,3210,Drama,Fast Times at Ridgemont High
274,3210,Romance,Fast Times at Ridgemont High
275,26566,Comedy,The Best of Times


In [73]:
# export genres to csv
path = '/Users/Kevin/cage-search/data/cleaned/cage_genres.csv'
genres.to_csv(path, index = False)

## create ratings dataset with rating, userId, associated movie

In [82]:
# filter ratings to only cage movies by merging with cage df
ratings.shape #(25,000,095, 4)
keys = list(cage['movieId'].values) # create list of Cage movieId
cage_ratings = ratings[ratings.movieId.isin(keys)] # filter to only cage movies
cage_ratings.shape #(262,762, 4)

(262762, 4)

In [83]:
ratings_cage.head()

,userId,movieId,rating,timestamp
95,2,733,4.5,1141415905
191,2,4023,3.0,1141417651
369,3,4023,4.0,1439473162
487,3,6708,4.0,1566090171
572,3,36529,4.0,1453904094


In [84]:
# change movieId to int in case
cage_ratings.movieId = cage_ratings.movieId.astype(int)

# import the names of movies to cage_ratings
cage_ratings = pd.merge(cage_ratings, cage[['movieId','movie', 'rottentomatoes']], on = 'movieId', how = 'left')

# drop the timestamp column (for now)
cage_ratings = cage_ratings.drop(columns = ['timestamp'])

cage_ratings.head()

,userId,movieId,rating,Movie,RottenTomatoes
0,2,733,4.5,The Rock,66
1,2,4023,3.0,The Family Man,53
2,3,4023,4.0,The Family Man,53
3,3,6708,4.0,Matchstick Men,82
4,3,36529,4.0,Lord of War,62


In [85]:
# double check dtypes
cage_ratings.dtypes

userId              int64
movieId             int64
rating            float64
Movie              object
RottenTomatoes      int64
dtype: object

In [87]:
# export cage ratings to csv
path = '/Users/Kevin/cage-search/data/cleaned/cage_ratings.csv'
cage_ratings.to_csv(path, index = False)

## Explore and merge the genome datasets

In [37]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [38]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [92]:
# change column types to predictable types - may be unnecessary
genome_scores.movieId = genome_scores.movieId.astype(int)
genome_scores.tagId = genome_scores.tagId.astype(int)
genome_tags.tagId = genome_tags.tagId.astype(int)
genome_tags.tag = genome_tags.tag.astype(str)

# rename the tags column as genome_tags not to get mixed up with tags column from tags.csv
genome_tags = genome_tags.rename(columns ={"tag": "genome_tag"})

# merge cage with genome_scores to get associated genome_tags
cage_genome = pd.merge(cage[['movieId', 'movie']], genome_scores, on = 'movieId', how = 'inner')

# merge with genome_tags to get tag descriptors
cage_genome = pd.merge(cage_genome, genome_tags[['tagId', 'genome_tag']], on = 'tagId', how = 'inner')

# rearrange columns to have relevance at the end
cage_genome = cage_genome[['movieId', 'movie', 'tagId', 'genome_tag', 'relevance']]

cage_genome

In [98]:
# check the largest values for relevance to see if they match up well with the movies
cage_genome.nlargest(10, 'relevance') # looks pretty accurate

,movieId,Movie,tagId,genome_tag,relevance
35587,84942,Drive Angry,457,goretastic,1.00000
81393,8972,National Treasure,1044,treasure hunt,0.99975
17464,1394,Raising Arizona,224,coen bros,0.99950
8378,51077,Ghost Rider,108,based on a comic,0.99925
81383,56775,National Treasure: Book of Secrets,1044,treasure hunt,0.99925
69870,6992,Guarding Tess,896,secret service,0.99900
76940,51077,Ghost Rider,987,super hero,0.99850
14398,3717,Gone in 60 Seconds,185,cars,0.99825
48115,7044,Wild at Heart,617,lynch,0.99825
77086,76251,Kick-Ass,989,superhero,0.99800


In [100]:
# export cage_genome to csv
path = '/Users/Kevin/cage-search/data/cleaned/cage_genome.csv'
cage_genome.to_csv(path, index = False)

## create user tags dataset

In [101]:
tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [ ]:
# drop the timestamp column for now
tags = tags.drop(columns = ['timestamp'])

tags = pd.merge(tags, cage_movies[['movieId', 'movie']], on = 'movieId')